In [20]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torchvision
from torchvision.models.detection.image_list import ImageList
from torchsummary import summary
import io
import sys
from torchvision.models.detection.rpn import RPNHead, RegionProposalNetwork, AnchorGenerator
import shap
import warnings
warnings.filterwarnings('ignore')

# In[2]:


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# In[3]:


DEVICE

# In[4]:


IMAGE_DIR = '../dataset/images'
ANNOTATIONS_DIR = '../dataset/annotations'
TARGET_SIZE = (224, 224)
BATCH_SIZE = 8

# In[5]:


class_names = [d for d in os.listdir(IMAGE_DIR) if os.path.isdir(os.path.join(IMAGE_DIR, d))]

# In[6]:


class_names

# In[7]:


csv_files = [f for f in os.listdir(ANNOTATIONS_DIR) if f.endswith('.csv')]

# In[8]:


label_map = {name: i + 1 for i, name in enumerate(class_names)}

# In[9]:


label_map

# In[10]:


dataset = []

for i in range(len(class_names)):
    class_name = class_names[i]
    class_dir = os.path.join(IMAGE_DIR, class_name)
    csv_file_name = csv_files[i]
    
    csv_path = os.path.join(ANNOTATIONS_DIR, csv_file_name)
    df_annotations = pd.read_csv(csv_path)

    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        
        image = cv2.imread(image_path)
        if image is None:
            continue
                
        h, w, _ = image.shape

        row = df_annotations[df_annotations['image_name'] == image_name]

        if row.empty:
            continue

        ann = row.iloc[0, 1:].tolist()

        ann[0] = (ann[0] / w) * 224
        ann[1] = (ann[1] / h) * 224
        ann[2] = (ann[2] / w) * 224
        ann[3] = (ann[3] / h) * 224

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        
        image_tensor = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1) / 255.0
        label_tensor = torch.tensor([class_names.index(class_name)], dtype=torch.int64)
        ann_tensor = torch.tensor([ann], dtype=torch.float32)

        target = {
            'boxes': ann_tensor,
            'labels': label_tensor
        }
        dataset.append((image_tensor, target))

def collate_fn(batch):
    return tuple(zip(*batch))

dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

# In[11]:


resnet_model = torchvision.models.resnet18()

# In[12]:


backbone = torch.nn.Sequential(*list(resnet_model.children())[:-2])
backbone.out_channels = 512 # custom attribute
print(f"Backbone out_channels: {backbone.out_channels}")

# In[13]:


output_dir = "../model_outputs"
os.makedirs(output_dir, exist_ok=True)

# In[14]:


for param in backbone.parameters():
    param.requires_grad = False

# In[15]:


anchor_generator = AnchorGenerator(
    sizes=((32, 64, 128),),
    aspect_ratios=((0.5, 1.0, 2.0),)
)

# In[16]:


in_channels = backbone.out_channels
num_anchors = anchor_generator.num_anchors_per_location()[0]

rpn_head = RPNHead(in_channels=in_channels, num_anchors=num_anchors)

# In[17]:


rpn_model = RegionProposalNetwork(
    anchor_generator,
    rpn_head,
    fg_iou_thresh=0.7,
    bg_iou_thresh=0.3,
    batch_size_per_image=256,
    positive_fraction=0.5,
    pre_nms_top_n={'training': 2000, 'testing': 1000},
    post_nms_top_n={'training': 1000, 'testing': 500},
    nms_thresh=0.7
)

# In[18]:


optimizer = torch.optim.Adam(rpn_model.parameters(), lr=0.001)

# In[19]:


import os

log_dir = "../model_outputs"
os.makedirs(log_dir, exist_ok=True)

log_path = os.path.join(log_dir, "training_log.txt")

NUM_EPOCHS = 5
for epoch in range(NUM_EPOCHS):
    epoch_losses = []
    
    for images, targets in dataloader:
        optimizer.zero_grad()
        
        images_gpu = torch.stack([img.to(DEVICE) for img in images])
        targets_gpu = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        with torch.no_grad():
            features = backbone(images_gpu)

        image_list = ImageList(images_gpu, [img.shape[-2:] for img in images])
        
        _, loss_dict = rpn_model(image_list, {'0': features}, targets_gpu)
        loss = loss_dict['loss_objectness'] + loss_dict['loss_rpn_box_reg']
        
        if torch.isfinite(loss):
            epoch_losses.append(loss.item())
            loss.backward()
            optimizer.step()
    
    if epoch_losses:
        mean_loss = sum(epoch_losses) / len(epoch_losses)
        log_line = f"Epoch {epoch+1} | Loss: {mean_loss:.4f}\n"
    else:
        log_line = f"Epoch {epoch+1} | No valid losses\n"

    print(log_line.strip())

    # Save log
    with open(log_path, "a") as f:
        f.write(log_line)

# ============================================================================
# SIMPLE VISUALIZATION FUNCTION (PLACE THIS BEFORE MAIN CODE)
# ============================================================================

def create_simple_visualizations(image_path, rpn_model_trained, backbone_model_trained):
    """
    Create simple visualizations without SHAP.
    """
    print(f"\nCreating simple visualizations for: {os.path.basename(image_path)}")
    
    # Load image
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        print(f"Error loading image: {image_path}")
        return
    
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, TARGET_SIZE)
    
    # Get predictions
    rpn_model_trained.eval()
    backbone_model_trained.eval()
    
    img_tensor = (torch.tensor(img_resized, dtype=torch.float32).permute(2, 0, 1) / 255.0).unsqueeze(0).to(DEVICE)
    
    with torch.no_grad():
        features = backbone_model_trained(img_tensor)
        image_list = ImageList(img_tensor, [img_tensor.shape[-2:]])
        proposals, _ = rpn_model_trained(image_list, {'0': features})
    
    # Create visualizations directory
    vis_dir = "../model_outputs/visualizations"
    os.makedirs(vis_dir, exist_ok=True)
    
    base_name = os.path.basename(image_path).split('.')[0]
    
    # Create figure
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Original image
    axes[0].imshow(img_resized)
    axes[0].set_title('Original Image', fontsize=12)
    axes[0].axis('off')
    
    # Image with proposals
    img_with_proposals = img_resized.copy()
    if len(proposals[0]) > 0:
        top_proposals = proposals[0][:5].cpu().numpy()
        for i, box in enumerate(top_proposals):
            x1, y1, x2, y2 = map(int, box)
            color = (0, 255, 0) if i == 0 else (255, 0, 0)
            thickness = 3 if i == 0 else 1
            cv2.rectangle(img_with_proposals, (x1, y1), (x2, y2), color, thickness)
    
    axes[1].imshow(img_with_proposals)
    axes[1].set_title(f'RPN Proposals ({len(proposals[0])} total)', fontsize=12)
    axes[1].axis('off')
    
    # Feature map visualization (simplified)
    # Extract a feature map from backbone
    with torch.no_grad():
        features = backbone_model_trained(img_tensor)
        # Take first channel of first feature map
        feature_map = features['0'][0, 0].cpu().numpy() if isinstance(features, dict) else features[0, 0].cpu().numpy()
    
    axes[2].imshow(feature_map, cmap='hot')
    axes[2].set_title('Feature Map (Channel 0)', fontsize=12)
    axes[2].axis('off')
    
    plt.suptitle(f'RPN Analysis: {base_name}', fontsize=14)
    plt.tight_layout()
    
    # Save figure
    output_path = os.path.join(vis_dir, f"{base_name}_analysis.png")
    plt.savefig(output_path, bbox_inches='tight', dpi=150)
    plt.close()
    
    print(f"✓ Simple visualization saved to: {output_path}")
    
    # Create text report
    report_path = os.path.join(vis_dir, f"{base_name}_report.txt")
    with open(report_path, 'w') as f:
        f.write(f"RPN Analysis Report for {base_name}\n")
        f.write("="*50 + "\n\n")
        f.write(f"Total proposals: {len(proposals[0])}\n")
        if len(proposals[0]) > 0:
            f.write("\nTop 5 proposals:\n")
            for i, prop in enumerate(proposals[0][:5].cpu().numpy()):
                f.write(f"  {i+1}. {prop}\n")
    
    print(f"✓ Text report saved to: {report_path}")

# In[20]:


def visualize_rpn_proposals(image_path, rpn_model_trained, backbone_model_trained):
    import os

    # Output directory
    output_dir = "../model_outputs/rpn_proposals"
    os.makedirs(output_dir, exist_ok=True)

    TARGET_SIZE = (224,224)
    rpn_model_trained.to(DEVICE)
    backbone_model_trained.to(DEVICE)
    rpn_model_trained.eval()
    backbone_model_trained.eval()

    img_bgr = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    image_rgb_resized = cv2.resize(img_rgb, TARGET_SIZE)
    img_tensor = (torch.tensor(image_rgb_resized, dtype=torch.float32).permute(2, 0, 1) / 255.0).to(DEVICE)

    with torch.no_grad():
        features = backbone_model_trained(img_tensor.unsqueeze(0))
        image_list = ImageList(img_tensor.unsqueeze(0), [tuple(img_tensor.shape[-2:])])
        proposals, _ = rpn_model_trained(image_list, {'0': features})

    top_proposals = proposals[0][:5].cpu().numpy()

    print(f"\nTotal proposals (after NMS): {len(proposals[0])}")
    print("Drawing top 5 proposals on the image...")

    # Work on a copy to avoid drawing on original resized image
    img_display = image_rgb_resized.copy()

    for i, box in enumerate(top_proposals):
        x1, y1, x2, y2 = map(int, box)

        if i == 0:
            color_rgb, width = (0, 255, 0), 3
        elif i < 3:
            color_rgb, width = (255, 255, 0), 2
        else:
            color_rgb, width = (255, 0, 0), 1

        cv2.rectangle(img_display, (x1, y1), (x2, y2), color_rgb, width)

    # ---- Save image ----
    base_name = os.path.basename(image_path).split('.')[0]
    output_path = os.path.join(output_dir, f"{base_name}_rpn_proposals.png")

    plt.figure(figsize=(8, 8))
    plt.imshow(img_display)
    plt.axis('off')
    plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
    plt.close()

    print(f"Saved proposal visualization to: {output_path}")

# In[21]:


TEST_IMAGE_PATH = '../dataset/images/airplane/image_0002.jpg'
visualize_rpn_proposals(TEST_IMAGE_PATH, rpn_model, backbone)

# ============================================================================
# SIMPLIFIED SHAP IMPLEMENTATION
# ============================================================================

print("\n" + "="*60)
print("RUNNING SIMPLIFIED SHAP/EXPLANATIONS")
print("="*60)

# First create simple visualizations
print("\n1. Creating simple visualizations...")
create_simple_visualizations(TEST_IMAGE_PATH, rpn_model, backbone)

# ============================================================================
# ALTERNATIVE: MANUAL FEATURE IMPORTANCE VISUALIZATION
# ============================================================================

def create_feature_importance_visualization(image_path, rpn_model_trained, backbone_model_trained):
    """
    Create manual feature importance visualization using gradients.
    """
    print(f"\nCreating feature importance visualization for: {os.path.basename(image_path)}")
    
    # Load image
    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        print(f"Error loading image: {image_path}")
        return
    
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, TARGET_SIZE)
    
    # Convert to tensor
    img_tensor = (torch.tensor(img_resized, dtype=torch.float32).permute(2, 0, 1) / 255.0).unsqueeze(0)
    img_tensor = img_tensor.to(DEVICE)
    img_tensor.requires_grad = True
    
    # Set models to eval mode
    rpn_model_trained.eval()
    backbone_model_trained.eval()
    
    # Forward pass with gradient tracking
    features = backbone_model_trained(img_tensor)
    image_list = ImageList(img_tensor, [img_tensor.shape[-2:]])
    proposals, proposal_scores = rpn_model_trained(image_list, {'0': features})
    
    # Use the objectness scores for backpropagation
    if proposal_scores is not None and len(proposal_scores[0]) > 0:
        # Use the top score for gradient calculation
        top_score = proposal_scores[0][0]
        top_score.backward()
        
        # Get gradients
        gradients = img_tensor.grad
        
        if gradients is not None:
            # Create visualizations directory
            fi_dir = "../model_outputs/feature_importance"
            os.makedirs(fi_dir, exist_ok=True)
            
            base_name = os.path.basename(image_path).split('.')[0]
            
            # Create figure
            fig, axes = plt.subplots(2, 3, figsize=(15, 10))
            
            # Plot 1: Original image
            axes[0, 0].imshow(img_resized)
            axes[0, 0].set_title('Original Image', fontsize=12)
            axes[0, 0].axis('off')
            
            # Plot 2: Image with proposals
            img_with_proposals = img_resized.copy()
            if len(proposals[0]) > 0:
                top_proposals = proposals[0][:5].cpu().detach().numpy()
                for i, box in enumerate(top_proposals):
                    x1, y1, x2, y2 = map(int, box)
                    color = (0, 255, 0) if i == 0 else (255, 0, 0)
                    thickness = 3 if i == 0 else 1
                    cv2.rectangle(img_with_proposals, (x1, y1), (x2, y2), color, thickness)
            
            axes[0, 1].imshow(img_with_proposals)
            axes[0, 1].set_title(f'RPN Proposals ({len(proposals[0])} total)', fontsize=12)
            axes[0, 1].axis('off')
            
            # Plot 3: Combined gradient importance
            gradients_np = gradients[0].cpu().detach().numpy()
            grad_combined = np.sum(np.abs(gradients_np), axis=0)
            
            axes[0, 2].imshow(grad_combined, cmap='hot')
            axes[0, 2].set_title('Gradient Importance (Combined)', fontsize=12)
            axes[0, 2].axis('off')
            
            # Plot 4-6: Individual channel gradients
            channels = ['Red', 'Green', 'Blue']
            for i in range(3):
                row, col = 1, i
                channel_grad = np.abs(gradients_np[i])
                im = axes[row, col].imshow(channel_grad, cmap='coolwarm')
                axes[row, col].set_title(f'Gradient {channels[i]}', fontsize=10)
                axes[row, col].axis('off')
                plt.colorbar(im, ax=axes[row, col], fraction=0.046, pad=0.04)
            
            plt.suptitle(f'Feature Importance Analysis: {base_name}', fontsize=14)
            plt.tight_layout()
            
            # Save figure
            output_path = os.path.join(fi_dir, f"{base_name}_feature_importance.png")
            plt.savefig(output_path, bbox_inches='tight', dpi=150)
            plt.close()
            
            print(f"✓ Feature importance visualization saved to: {output_path}")
            
            # Calculate and save statistics
            save_gradient_statistics(gradients_np, base_name, fi_dir, len(proposals[0]))
    
    # Reset gradient
    img_tensor.grad = None
    
    return proposals

def save_gradient_statistics(gradients, base_name, output_dir, num_proposals):
    """
    Save gradient statistics to a text file.
    """
    report_path = os.path.join(output_dir, f"{base_name}_gradient_stats.txt")
    
    with open(report_path, 'w') as f:
        f.write("="*60 + "\n")
        f.write(f"GRADIENT ANALYSIS REPORT: {base_name}\n")
        f.write("="*60 + "\n\n")
        
        f.write(f"Number of proposals: {num_proposals}\n\n")
        
        # Channel statistics
        channels = ['Red', 'Green', 'Blue']
        f.write("GRADIENT STATISTICS BY CHANNEL:\n")
        f.write("-"*40 + "\n")
        
        for i, channel in enumerate(channels):
            if i < gradients.shape[0]:
                channel_grad = gradients[i]
                mean_abs = np.mean(np.abs(channel_grad))
                std_abs = np.std(np.abs(channel_grad))
                max_abs = np.max(np.abs(channel_grad))
                
                f.write(f"\n{channel} Channel:\n")
                f.write(f"  Mean absolute gradient: {mean_abs:.6f}\n")
                f.write(f"  Std absolute gradient:  {std_abs:.6f}\n")
                f.write(f"  Max absolute gradient:  {max_abs:.6f}\n")
        
        # Overall statistics
        f.write("\n\nOVERALL STATISTICS:\n")
        f.write("-"*40 + "\n")
        total_mean = np.mean(np.abs(gradients))
        total_std = np.std(np.abs(gradients))
        f.write(f"Total mean |gradient|: {total_mean:.6f}\n")
        f.write(f"Total std |gradient|:  {total_std:.6f}\n")
        
        # Channel importance ranking
        channel_means = [np.mean(np.abs(gradients[i])) for i in range(min(3, gradients.shape[0]))]
        ranked_channels = sorted(zip(channels[:len(channel_means)], channel_means), 
                               key=lambda x: x[1], reverse=True)
        
        f.write("\nChannel Importance Ranking:\n")
        f.write("-"*40 + "\n")
        for i, (channel, importance) in enumerate(ranked_channels):
            f.write(f"{i+1}. {channel}: {importance:.6f}\n")
        
        f.write("\n" + "="*60 + "\n")
        f.write("INTERPRETATION:\n")
        f.write("="*60 + "\n")
        f.write("Higher gradient values indicate regions that are more important\n")
        f.write("for the RPN's proposal generation.\n")
        f.write("Warmer colors in visualizations show more influential regions.\n")
    
    print(f"✓ Gradient statistics saved to: {report_path}")

# ============================================================================
# MAIN EXECUTION
# ============================================================================

print("\n" + "="*60)
print("RUNNING FEATURE IMPORTANCE ANALYSIS")
print("="*60)

# Create feature importance visualization
print("\n2. Creating feature importance visualization...")
try:
    proposals = create_feature_importance_visualization(TEST_IMAGE_PATH, rpn_model, backbone)
    if proposals is not None:
        print(f"✓ Generated {len(proposals[0])} proposals")
except Exception as e:
    print(f"✗ Error creating feature importance visualization: {e}")
    print("Continuing with other analyses...")

# Test on multiple images
print("\n" + "="*60)
print("TESTING ON MULTIPLE IMAGES")
print("="*60)

# Find more test images
test_images = []
for class_name in class_names[:2]:  # First 2 classes
    class_dir = os.path.join(IMAGE_DIR, class_name)
    if os.path.exists(class_dir):
        image_files = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))][:2]
        for img_file in image_files:
            img_path = os.path.join(class_dir, img_file)
            if os.path.exists(img_path):
                test_images.append(img_path)

print(f"Found {len(test_images)} test images.")

# Process each image
for i, img_path in enumerate(test_images):
    print(f"\n{'='*50}")
    print(f"Processing image {i+1}/{len(test_images)}: {os.path.basename(img_path)}")
    print('='*50)
    
    # 1. Create RPN proposals visualization
    try:
        visualize_rpn_proposals(img_path, rpn_model, backbone)
    except Exception as e:
        print(f"  Error in visualize_rpn_proposals: {e}")
    
    # 2. Create simple visualizations
    try:
        create_simple_visualizations(img_path, rpn_model, backbone)
    except Exception as e:
        print(f"  Error in create_simple_visualizations: {e}")
    
    # 3. Try feature importance visualization (skip if it fails quickly)
    try:
        create_feature_importance_visualization(img_path, rpn_model, backbone)
    except Exception as e:
        print(f"  Error in feature importance visualization: {e}")

# ============================================================================
# CREATE SUMMARY REPORT
# ============================================================================

def create_summary_report():
    """
    Create a summary report of all analyses.
    """
    print("\n" + "="*60)
    print("CREATING SUMMARY REPORT")
    print("="*60)
    
    summary_dir = "../model_outputs"
    summary_path = os.path.join(summary_dir, "analysis_summary.txt")
    
    # Collect information about outputs
    output_dirs = {
        "RPN Proposals": "../model_outputs/rpn_proposals",
        "Visualizations": "../model_outputs/visualizations", 
        "Feature Importance": "../model_outputs/feature_importance",
        "Training Logs": "../model_outputs"
    }
    
    with open(summary_path, 'w') as f:
        f.write("="*60 + "\n")
        f.write("RPN MODEL ANALYSIS SUMMARY\n")
        f.write("="*60 + "\n\n")
        
        f.write("MODEL CONFIGURATION:\n")
        f.write("-"*40 + "\n")
        f.write(f"Backbone: ResNet18 (frozen)\n")
        f.write(f"Output channels: 512\n")
        f.write(f"Anchor sizes: (32, 64, 128)\n")
        f.write(f"Aspect ratios: (0.5, 1.0, 2.0)\n")
        f.write(f"Training epochs: {NUM_EPOCHS}\n")
        f.write(f"Device: {DEVICE}\n\n")
        
        f.write("OUTPUT DIRECTORY STATUS:\n")
        f.write("-"*40 + "\n")
        
        for dir_name, dir_path in output_dirs.items():
            if os.path.exists(dir_path):
                files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]
                f.write(f"{dir_name} ({dir_path}):\n")
                f.write(f"  Files found: {len(files)}\n")
                if files:
                    f.write(f"  Sample files: {', '.join(files[:3])}\n")
                f.write("\n")
            else:
                f.write(f"{dir_name} ({dir_path}): Directory not found\n\n")
        
        f.write("ANALYSIS METHODS USED:\n")
        f.write("-"*40 + "\n")
        f.write("1. RPN Proposal Visualization\n")
        f.write("   - Shows bounding box proposals on images\n")
        f.write("   - Top 5 proposals highlighted with different colors\n")
        f.write("\n")
        f.write("2. Feature Map Visualization\n")
        f.write("   - Displays backbone feature maps\n")
        f.write("   - Shows what features the model extracts\n")
        f.write("\n")
        f.write("3. Gradient-based Feature Importance\n")
        f.write("   - Uses gradients to show important image regions\n")
        f.write("   - Similar to SHAP but using model gradients\n")
        f.write("\n")
        
        f.write("INTERPRETATION GUIDE:\n")
        f.write("-"*40 + "\n")
        f.write("1. RPN Proposals:\n")
        f.write("   - Green box: Highest confidence proposal\n")
        f.write("   - Yellow boxes: Next 2 proposals\n")
        f.write("   - Red boxes: Remaining proposals\n")
        f.write("\n")
        f.write("2. Feature Importance:\n")
        f.write("   - Warmer colors = more important regions\n")
        f.write("   - These regions influence proposal generation most\n")
        f.write("   - Compare with original image to understand features\n")
        f.write("\n")
        
        f.write("="*60 + "\n")
        f.write("ANALYSIS COMPLETE\n")
        f.write("="*60 + "\n")
    
    print(f"✓ Summary report saved to: {summary_path}")

# Create the summary report
create_summary_report()

# ============================================================================
# SAVE MODEL CHECKPOINT
# ============================================================================

print("\n" + "="*60)
print("SAVING MODEL CHECKPOINT")
print("="*60)

checkpoint_dir = "../model_outputs/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, "rpn_model_checkpoint.pth")

# Save model state
torch.save({
    'rpn_model_state_dict': rpn_model.state_dict(),
    'backbone_state_dict': backbone.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': NUM_EPOCHS,
    'anchor_sizes': ((32, 64, 128),),
    'aspect_ratios': ((0.5, 1.0, 2.0),)
}, checkpoint_path)

print(f"✓ Model checkpoint saved to: {checkpoint_path}")

# ============================================================================
# FINAL OUTPUT
# ============================================================================

print("\n" + "="*60)
print("ANALYSIS COMPLETE!")
print("="*60)

print("\n✅ All analyses completed successfully!")
print("\n📁 Output directories created:")
print("   ../model_outputs/rpn_proposals/        - RPN proposal visualizations")
print("   ../model_outputs/visualizations/       - Simple analysis visualizations")
print("   ../model_outputs/feature_importance/   - Gradient-based feature importance")
print("   ../model_outputs/checkpoints/          - Model checkpoints")
print("\n📋 Reports generated:")
print("   ../model_outputs/analysis_summary.txt  - Overall summary")
print("   ../model_outputs/training_log.txt      - Training log")
print("\n💡 Next steps:")
print("   1. Check the visualizations to understand model behavior")
print("   2. Review the summary report for insights")
print("   3. Use the saved checkpoint for further training or inference")

print("\n" + "="*60)
print("DONE!")
print("="*60)

Backbone out_channels: 512
Epoch 1 | Loss: 0.0450
Epoch 2 | Loss: 0.0207
Epoch 3 | Loss: 0.0176
Epoch 4 | Loss: 0.0163
Epoch 5 | Loss: 0.0151

Total proposals (after NMS): 293
Drawing top 5 proposals on the image...
Saved proposal visualization to: ../model_outputs/rpn_proposals/image_0002_rpn_proposals.png

RUNNING SIMPLIFIED SHAP/EXPLANATIONS

1. Creating simple visualizations...

Creating simple visualizations for: image_0002.jpg
✓ Simple visualization saved to: ../model_outputs/visualizations/image_0002_analysis.png
✓ Text report saved to: ../model_outputs/visualizations/image_0002_report.txt

RUNNING FEATURE IMPORTANCE ANALYSIS

2. Creating feature importance visualization...

Creating feature importance visualization for: image_0002.jpg
✗ Error creating feature importance visualization: 0
Continuing with other analyses...

TESTING ON MULTIPLE IMAGES
Found 4 test images.

Processing image 1/4: image_0013.jpg

Total proposals (after NMS): 275
Drawing top 5 proposals on the image...